### RSI for Nifty200

In [193]:
import os
import datetime
import pandas as pd
import numpy as np
import pandas_datareader.data as web
idx = pd.IndexSlice

In [194]:
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
import investpy
import pyfolio as pf
import backtrader as bt
import collections
import pymysql

In [195]:
import warnings
warnings.filterwarnings("ignore")
sns.set_style('whitegrid')
np.random.seed(42)
bt.__version__

'1.9.76.123'

In [196]:
BENCHMARK_TICKER = 'NIFTY 100'
START_DATE = datetime.datetime(2010, 1, 1)
END_DATE = datetime.datetime(2020, 12, 31)

#### Prepare Data
We will take top Nifty 100 Indian companies for this analysis.

In [197]:
DATA_STORE = '../../Data-Daily/india_asset.h5'

In [198]:
with pd.HDFStore(DATA_STORE) as store:
    print(store.info())

<class 'pandas.io.pytables.HDFStore'>
File path: ../../Data-Daily/india_asset.h5
/bse/bse50/metadata                             frame        (shape->[49,6])     
/bse/bse500/metadata                            frame        (shape->[472,7])    
/ind/bse500/prices                              frame        (shape->[1707187,7])
/ind/equity/prices                              frame        (shape->[1755502,7])
/ind/investing_meta/metadata                    frame        (shape->[1,5])      
/ind/nifty500_investing/prices                  frame        (shape->[1471831,5])
/ind/nifty50_investing/prices                   frame        (shape->[213669,5]) 
/mf_eft/metadata                                frame        (shape->[23,5])     
/nse/nifty100/metadata                          frame        (shape->[1,10])     
/nse/nifty100_mid/metadata                      frame        (shape->[1,4])      
/nse/nifty100_small/metadata                    frame        (shape->[1,4])      
/nse/nifty200/met

In [199]:
with pd.HDFStore(DATA_STORE) as store:
    nifty200_m = (store['/nse/nifty200/metadata'])
nifty200_m.head(2)

,key_0,Symbol,Company Name,Industry,Series,country,name,full_name,currency,symbol
isin,,,,,,,,,,
INE012A01025,INE012A01025,ACC,ACC Ltd.,CEMENT & CEMENT PRODUCTS,EQ,india,ACC,ACC Ltd,INR,ACC
INE949L01017,INE949L01017,AUBANK,AU Small Finance Bank Ltd.,FINANCIAL SERVICES,EQ,india,AU Small Finance Bank,AU Small Finance Bank Ltd,INR,AUFI


In [200]:
tickers = nifty200_m.dropna().symbol.to_list()
# tickers = tickers[:10]
print(tickers)
print(len(tickers))

['ACC', 'AUFI', 'ARTI', 'ABOT', 'ADEL', 'ADAG', 'ADNA', 'APSE', 'ADAI', 'ADTB', 'ADIA', 'AJPH', 'ALEM', 'ALKE', 'AMAR', 'ABUJ', 'APLH', 'APLO', 'ASOK', 'ASPN', 'ARBN', 'AVEU', 'AXBK', 'BAJA', 'BJFN', 'BJFS', 'BJAT', 'BLKI', 'BANH', 'BOB', 'BOI', 'BATA', 'BRGR', 'BAJE', 'BFRG', 'BHEL', 'BPCL', 'BRTI', 'BHRI', 'BION', 'BBRM', 'BOSH', 'BRIT', 'CESC', 'CADI', 'CNBK', 'CAST', 'CHLA', 'CIPL', 'CTBK', 'COAL', 'NITT', 'COLG', 'CCRI', 'CORF', 'CROP', 'CUMM', 'DLF', 'DABU', 'DALB', 'INDB', 'DIVI', 'DLPA', 'REDY', 'EDEL', 'EMAM', 'ENDU', 'ESCO', 'EXID', 'FED', 'FOHE', 'FRTL', 'GAIL', 'GMRI', 'GENA', 'GLEN', 'GODE', 'GOCP', 'GODI', 'GODR', 'GRAS', 'GGAS', 'GSPT', 'HCLT', 'HDFA', 'HDBK', 'HDFL', 'HVEL', 'HROM', 'HALC', 'HPCL', 'HLL', 'HZNC', 'HUDC', 'HDFC', 'ICBK', 'ICIL', 'ICIR', 'ICCI', 'IDFB', 'ITC', 'INBF', 'IHTL', 'IOC', 'INIR', 'IGAS', 'INBK', 'INED', 'INFY', 'INGL', 'IPCA', 'JSWE', 'JSTL', 'JNSP', 'JUBI', 'KTKM', 'LTFH', 'LTEH', 'LICH', 'LRTI', 'LART', 'LUPN', 'MRF', 'MGAS', 'MMFS', 'MAHM', 

In [201]:
# tickers = ['ACC', 'AUFI', 'ARTI', 'ABOT', 'ADEL', 'ADAG', 'ADNA', 'APSE', 'ADAI', 'ADTB', 'ADIA', 'AJPH', 'ALEM', 'ALKE', 
#            'AMAR', 'ABUJ', 'APLH', 'APLO', 'ASOK', 'ASPN', 'ARBN', 'AVEU', 'AXBK', 'BAJA', 'BJFN', 'BJFS', 'BJAT', 'BLKI', 
#            'BANH', 'BOB', 'BOI', 'BATA', 'BRGR', 'BAJE', 'BFRG', 'BHEL', 'BPCL', 'BRTI', 'BHRI', 'BION', 'BBRM', 'BOSH', 
#            'BRIT', 'CESC', 'CADI', 'CNBK', 'CAST', 'CHLA', 'CIPL', 'CTBK', 'COAL', 'NITT', 'COLG', 'CCRI', 'CORF', 'CROP', 
#            'CUMM', 'DLF', 'DABU', 'DALB', 'INDB', 'DIVI', 'DLPA', 'REDY', 'EDEL', 'EMAM', 'ENDU', 'ESCO', 'EXID', 'FED', 
#            'FOHE', 
#            'FRTL', 'GAIL', 'GMRI', 'GENA', 'GLEN', 'GODE', 'GOCP', 'GODI', 'GODR', 'GRAS', 'GGAS', 'GSPT', 'HCLT', 'HDFA', 
#            'HDBK', 'HDFL', 'HVEL', 'HROM', 'HALC', 'HPCL', 'HLL', 'HZNC', 'HUDC', 'HDFC', 'ICBK', 'ICIL', 'ICIR', 'ICCI', 
#            'IDFB', 'ITC', 'INBF', 'IHTL', 'IOC', 'INIR', 'IGAS', 'INBK', 'INED', 'INFY', 'INGL', 'IPCA', 'JSWE', 'JSTL', 
#            'JNSP', 'JUBI', 'KTKM', 'LTFH', 'LTEH', 'LICH', 'LRTI', 'LART', 'LUPN', 'MRF', 'MGAS', 'MMFS', 'MAHM', 'MNFL', 
#            'MRCO', 'MRTI', 'MAXI', 'MINT', 'MOSS', 'MBFL', 'MUTT', 'NATP', 'NMDC', 'NTPC', 'NALU', 'NAFL', 'NEST', 'RELL', 
#            'OEBO', 'ONGC', 'OILI', 'ORCL', 'PIIL', 'PAGE', 'PLNG', 'PFIZ', 'PIDI', 'PIRA', 'POLC', 'PWFC', 'PGRD', 'PREG', 
#            'PROC', 'PNBK', 'RATB', 'RECM', 'REXP', 'RELI', 'SBIL', 'SRFL', 'SANO', 'SHCM', 'SRTR', 'SIEM', 'SBI', 'SAIL', 
#            'SUN', 'SUTV', 'SYNN', 'TVSM', 'TTCH', 'TCS', 'TAGL', 'TAMO', 'TTPW', 'TISC', 'TEML', 'TRCE', 'TITN', 'TORP', 
#            'TOPO', 'TREN', 'UPLL', 'ULTC', 'UNBK', 'UBBW', 'UNSP', 'VGUA', 'VARB', 'VODA', 'VOLT', 'WHIR', 'WIPR', 'ZEE']

In [202]:
# def get_db_conn(p_uid='', p_pwd='', p_host='', p_database='tradesoft', p_port=3306):
#     pconn = pymysql.connect( user=p_uid, password=p_pwd, host=p_host, database=p_database, port=p_port )
#     pconn.autocommit = False
#     return pconn

In [203]:
# def historical_price(scripts, db_conn):
#     columns = ['close','high','low','open','volume']
#     fmt='%Y%m%d %H:%M:%S'
#     def data(ticker):
#         print(f'Processing...{ticker}')
#         v_sql_stmt = f"SELECT * FROM t_quotes_eod WHERE ticker = '{ticker}'" 
#         df_quotes = pd.read_sql(v_sql_stmt, db_conn, parse_dates={'trade_date':fmt})
#         df_quotes = df_quotes.set_index('trade_date')
#         df_quotes.index.name = 'date'
#         df_quotes = df_quotes[columns]
#         df_quotes = df_quotes.sort_values(by='date', ascending=True)
#         df_quotes = df_quotes[~df_quotes.index.duplicated()]
#         return df_quotes
#     datas = map(data, scripts)
#     return(pd.concat(datas, keys=tickers, names=['ticker', 'date']))

In [204]:
# db_conn = get_db_conn(p_uid='freedbtech_tradesoft', p_pwd='tradesoft', p_host='freedb.tech', 
#                       p_database='freedbtech_tradesoft', p_port=3306)

# prices = historical_price(tickers, db_conn)
# db_conn.close()

In [208]:
# prices.tail()

In [206]:
columns = ['close','high','low','open','volume']
with pd.HDFStore(DATA_STORE) as store:
    prices = store['/ind/nifty500_investing/prices'].loc[idx[tickers, :], columns]
prices.head(3)

close    high    low   open   volume
ticker date                                             
ACC    2000-01-03  268.55  271.45  255.1  258.9  5447617
       2000-01-04  256.60  269.50  255.1  268.0  2978528
       2000-01-05  250.30  268.00  244.5  249.9  2802364

In [209]:
# remove tickers where we have less than 10 years of data.
min_obs = 2520
nobs = prices.groupby(level='ticker').size()
keep = nobs[nobs>min_obs].index
prices = prices.loc[idx[keep,:], :]
prices.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 620630 entries, ('ABOT', Timestamp('2000-01-03 00:00:00')) to ('ZEE', Timestamp('2021-01-21 00:00:00'))
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   close   620630 non-null  float64
 1   high    620630 non-null  float64
 2   low     620630 non-null  float64
 3   open    620630 non-null  float64
 4   volume  620630 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 26.2+ MB


In [210]:
print(prices.index.unique(level='ticker'))

Index(['ABOT', 'ABUJ', 'ACC', 'ADEL', 'AMAR', 'APLH', 'APLO', 'APSE', 'ARBN',
       'ARTI',
       ...
       'ULTC', 'UNBK', 'UNSP', 'UPLL', 'VGUA', 'VODA', 'VOLT', 'WHIR', 'WIPR',
       'ZEE'],
      dtype='object', name='ticker', length=154)


In [211]:
class StrategyEqWt(bt.Strategy):
    params = dict(
        num_positions=30,
        rebalance_days = [1,4],
        reserve = 0.05,
        printlog = True,
    )
    
    def log(self, txt, dt=None, doprint=False):
        ''' Logging function '''
        if self.params.printlog or doprint:
            dt = dt or self.data.datetime[0]
            if isinstance(dt, float):
                dt = bt.num2date(dt)
            print("%s, %s" % (dt.isoformat(), txt))

    def __init__(self):
        self.inds = {}
        for d in self.datas:
            self.inds[d] = {}
            self.inds[d]['rsi'] = bt.ind.RSI(d, plot=False)
            
        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
            
    def next(self):
        dt = self.data.datetime.datetime()
        if dt.weekday() in self.p.rebalance_days:
            self.rebalance()

    def rebalance(self):
        ranked_sectors = []
        for d in self.datas:
            row = [d, d.p.name, self.inds[d]['rsi'][0]]
            ranked_sectors.append(row)
            
        sectors = sorted(ranked_sectors, key=lambda x: x[2], reverse=False)
#         for s in sectors:
#             self.log('Sectors List: %s | RSI: %s' % (s[1], s[2]))

        buys = [s[0] for s in sectors[:self.p.num_positions]]
        weight = 1 / len(buys)

        # prepare quick lookup list of stocks currently holding a position
        posdata = [d for d, pos in self.getpositions().items() if pos]

        # remove those no longer top ranked
        for d in (d for d in posdata if d not in buys):
            if self.inds[d]['rsi'][0] >= 70:
                if self.inds[d]['rsi'][-1] > self.inds[d]['rsi'][0] > self.inds[d]['rsi'][1]:
                    self.close(d)
    
        # rebalance those already top ranked and still there
        for d in (d for d in posdata if d in buys):
            cash = self.broker.get_cash()
            if cash <= 0:
                break
            self.order_target_percent(d, target=weight)
    
        # issue a target order for the newly top securities
        for d in buys:
            cash = self.broker.get_cash()
            if cash <= 0:
                break
            if self.inds[d]['rsi'][0] <= 30:
                if self.inds[d]['rsi'][-1] < self.inds[d]['rsi'][0] < self.inds[d]['rsi'][1]:
                    self.order_target_percent(d, target=weight)
            
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price, order.executed.value, order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price, order.executed.value,order.executed.comm))

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def stop(self):
        self.log('| %2d |  %.2f |' %
                 (self.p.num_positions, self.broker.getvalue()), doprint=True) 

In [220]:
# Model Settings
startcash = 500000
num_positions = 20
reserve = 0.05
printlog=True,
rebalance_days = [1,4]

# Commission and Slippage Settings
commission = 0.0025

In [221]:
cerebro = bt.Cerebro(stdstats=False)
cerebro.broker.setcash(startcash)
cerebro.broker.setcommission(commission=commission)

In [222]:
# Add securities as datas1:
for ticker, data in prices.groupby(level=0):
    if ticker in tickers:
        print(f"Adding ticker: {ticker}")
        data = bt.feeds.PandasData(dataname=data.droplevel(level=0),
                                   name=str(ticker),
                                   fromdate=START_DATE,
                                   todate=END_DATE,
                                   plot=False)
        cerebro.adddata(data)

Adding ticker: ABOT
Adding ticker: ABUJ
Adding ticker: ACC
Adding ticker: ADEL
Adding ticker: AMAR
Adding ticker: APLH
Adding ticker: APLO
Adding ticker: APSE
Adding ticker: ARBN
Adding ticker: ARTI
Adding ticker: ASOK
Adding ticker: ASPN
Adding ticker: AXBK
Adding ticker: BAJA
Adding ticker: BAJE
Adding ticker: BATA
Adding ticker: BBRM
Adding ticker: BFRG
Adding ticker: BHEL
Adding ticker: BION
Adding ticker: BJAT
Adding ticker: BJFN
Adding ticker: BJFS
Adding ticker: BLKI
Adding ticker: BOB
Adding ticker: BOI
Adding ticker: BOSH
Adding ticker: BPCL
Adding ticker: BRGR
Adding ticker: BRIT
Adding ticker: BRTI
Adding ticker: CADI
Adding ticker: CAST
Adding ticker: CCRI
Adding ticker: CESC
Adding ticker: CHLA
Adding ticker: CNBK
Adding ticker: COAL
Adding ticker: COLG
Adding ticker: CORF
Adding ticker: CTBK
Adding ticker: CUMM
Adding ticker: DABU
Adding ticker: DIVI
Adding ticker: DLF
Adding ticker: EDEL
Adding ticker: EMAM
Adding ticker: ESCO
Adding ticker: EXID
Adding ticker: FED
Addin

In [223]:
cerebro.addanalyzer(bt.analyzers.Returns)
cerebro.addanalyzer(bt.analyzers.DrawDown)
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

cerebro.addstrategy(StrategyEqWt,
                    num_positions = num_positions,
                    rebalance_days = rebalance_days,
                    reserve = reserve,
                    printlog = printlog,
                   )

0

In [224]:
%%time
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
# Run the strategy. Results will be output from stop.
results = cerebro.run()

# # Run over everything
# cerebro.run(maxcpus=1)

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 500000.00
2010-11-29T00:00:00, BUY EXECUTED, Price: 137.65, Cost: 25327.60, Comm 63.32
2010-11-29T00:00:00, BUY EXECUTED, Price: 311.28, Cost: 24902.40, Comm 62.26
2010-11-29T00:00:00, BUY EXECUTED, Price: 86.65, Cost: 25128.50, Comm 62.82
2010-12-01T00:00:00, SELL EXECUTED, Price: 87.25, Cost: 86.65, Comm 0.22
2010-12-01T00:00:00, BUY EXECUTED, Price: 175.53, Cost: 24749.73, Comm 61.87
2010-12-01T00:00:00, BUY EXECUTED, Price: 7900.00, Cost: 23700.00, Comm 59.25
2010-12-01T00:00:00, SELL EXECUTED, Price: 87.25, Cost: 86.65, Comm 0.22
2010-12-01T00:00:00, BUY EXECUTED, Price: 100.35, Cost: 24987.15, Comm 62.47
2010-12-01T00:00:00, BUY EXECUTED, Price: 92.04, Cost: 25126.92, Comm 62.82
2010-12-01T00:00:00, BUY EXECUTED, Price: 87.60, Cost: 26280.00, Comm 65.70
2010-12-01T00:00:00, BUY EXECUTED, Price: 91.01, Cost: 25300.78, Comm 63.25
2010-12-01T00:00:00, BUY EXECUTED, Price: 200.38, Cost: 24847.12, Comm 62.12
2010-12-06T00:00:00, BUY EXECUTED, Price: 78.00, Co

2011-03-21T00:00:00, BUY EXECUTED, Price: 72.60, Cost: 217.80, Comm 0.54
2011-03-21T00:00:00, BUY EXECUTED, Price: 107.20, Cost: 536.00, Comm 1.34
2011-03-23T00:00:00, BUY EXECUTED, Price: 71.50, Cost: 572.00, Comm 1.43
2011-03-23T00:00:00, SELL EXECUTED, Price: 110.50, Cost: 266.72, Comm 0.55
2011-03-28T00:00:00, Order Canceled/Margin/Rejected
2011-03-28T00:00:00, BUY EXECUTED, Price: 72.70, Cost: 218.10, Comm 0.55
2011-03-30T00:00:00, BUY EXECUTED, Price: 72.00, Cost: 720.00, Comm 1.80
2011-03-30T00:00:00, SELL EXECUTED, Price: 336.45, Cost: 1556.40, Comm 4.21
2011-04-04T00:00:00, Order Canceled/Margin/Rejected
2011-04-04T00:00:00, SELL EXECUTED, Price: 2050.20, Cost: 24500.00, Comm 71.76
2011-04-04T00:00:00, BUY EXECUTED, Price: 97.45, Cost: 25921.70, Comm 64.80
2011-04-06T00:00:00, BUY EXECUTED, Price: 97.31, Cost: 97.31, Comm 0.24
2011-04-06T00:00:00, SELL EXECUTED, Price: 99.50, Cost: 389.80, Comm 0.99
2011-04-06T00:00:00, SELL EXECUTED, Price: 124.45, Cost: 1600.30, Comm 3.73
20

2011-08-29T00:00:00, Order Canceled/Margin/Rejected
2011-08-29T00:00:00, Order Canceled/Margin/Rejected
2011-08-29T00:00:00, Order Canceled/Margin/Rejected
2011-09-02T00:00:00, Order Canceled/Margin/Rejected
2011-09-02T00:00:00, Order Canceled/Margin/Rejected
2011-09-02T00:00:00, Order Canceled/Margin/Rejected
2011-09-02T00:00:00, Order Canceled/Margin/Rejected
2011-09-02T00:00:00, Order Canceled/Margin/Rejected
2011-09-05T00:00:00, Order Canceled/Margin/Rejected
2011-09-05T00:00:00, Order Canceled/Margin/Rejected
2011-09-05T00:00:00, Order Canceled/Margin/Rejected
2011-09-05T00:00:00, Order Canceled/Margin/Rejected
2011-09-05T00:00:00, Order Canceled/Margin/Rejected
2011-09-07T00:00:00, Order Canceled/Margin/Rejected
2011-09-07T00:00:00, SELL EXECUTED, Price: 124.78, Cost: 1373.05, Comm 3.12
2011-09-12T00:00:00, BUY EXECUTED, Price: 28.20, Cost: 310.20, Comm 0.78
2011-09-12T00:00:00, SELL EXECUTED, Price: 80.56, Cost: 3853.32, Comm 8.66
2011-09-12T00:00:00, SELL EXECUTED, Price: 124.4

2012-01-11T00:00:00, BUY EXECUTED, Price: 79.22, Cost: 316.88, Comm 0.79
2012-01-11T00:00:00, BUY EXECUTED, Price: 54.49, Cost: 326.94, Comm 0.82
2012-01-11T00:00:00, BUY EXECUTED, Price: 85.02, Cost: 85.02, Comm 0.21
2012-01-16T00:00:00, BUY EXECUTED, Price: 83.83, Cost: 167.66, Comm 0.42
2012-01-23T00:00:00, BUY EXECUTED, Price: 83.83, Cost: 502.98, Comm 1.26
2012-01-25T00:00:00, SELL EXECUTED, Price: 134.85, Cost: 10207.84, Comm 39.11
2012-01-30T00:00:00, SELL EXECUTED, Price: 142.66, Cost: 26574.62, Comm 72.04
2012-01-30T00:00:00, BUY EXECUTED, Price: 133.40, Cost: 533.60, Comm 1.33
2012-02-01T00:00:00, SELL EXECUTED, Price: 76.95, Cost: 2908.88, Comm 5.00
2012-02-01T00:00:00, BUY EXECUTED, Price: 133.75, Cost: 133.75, Comm 0.33
2012-02-06T00:00:00, BUY EXECUTED, Price: 68.17, Cost: 1295.23, Comm 3.24
2012-02-08T00:00:00, BUY EXECUTED, Price: 134.30, Cost: 402.90, Comm 1.01
2012-02-15T00:00:00, BUY EXECUTED, Price: 68.75, Cost: 687.50, Comm 1.72
2012-02-15T00:00:00, BUY EXECUTED, P

2012-06-18T00:00:00, BUY EXECUTED, Price: 60.52, Cost: 726.24, Comm 1.82
2012-06-18T00:00:00, BUY EXECUTED, Price: 22.00, Cost: 572.00, Comm 1.43
2012-06-18T00:00:00, BUY EXECUTED, Price: 130.20, Cost: 130.20, Comm 0.33
2012-06-20T00:00:00, Order Canceled/Margin/Rejected
2012-06-20T00:00:00, Order Canceled/Margin/Rejected
2012-06-20T00:00:00, BUY EXECUTED, Price: 265.06, Cost: 795.18, Comm 1.99
2012-06-25T00:00:00, BUY EXECUTED, Price: 265.99, Cost: 1063.96, Comm 2.66
2012-06-25T00:00:00, BUY EXECUTED, Price: 76.00, Cost: 532.00, Comm 1.33
2012-06-27T00:00:00, BUY EXECUTED, Price: 254.08, Cost: 762.24, Comm 1.91
2012-07-02T00:00:00, SELL EXECUTED, Price: 31.25, Cost: 31467.79, Comm 86.80
2012-07-02T00:00:00, SELL EXECUTED, Price: 31.00, Cost: 3632.56, Comm 7.44
2012-07-02T00:00:00, BUY EXECUTED, Price: 246.46, Cost: 985.84, Comm 2.46
2012-07-02T00:00:00, BUY EXECUTED, Price: 134.85, Cost: 539.40, Comm 1.35
2012-07-09T00:00:00, BUY EXECUTED, Price: 317.00, Cost: 951.00, Comm 2.38
2012-0

...

2019-03-05T00:00:00, BUY EXECUTED, Price: 571.20, Cost: 2856.00, Comm 7.14
2019-03-05T00:00:00, BUY EXECUTED, Price: 18.06, Cost: 3286.92, Comm 8.22
2019-03-06T00:00:00, SELL EXECUTED, Price: 710.05, Cost: 19686.00, Comm 60.35
2019-03-06T00:00:00, BUY EXECUTED, Price: 570.20, Cost: 2280.80, Comm 5.70
2019-03-06T00:00:00, BUY EXECUTED, Price: 18.12, Cost: 2554.92, Comm 6.39
2019-03-11T00:00:00, BUY EXECUTED, Price: 705.70, Cost: 2117.10, Comm 5.29
2019-03-11T00:00:00, BUY EXECUTED, Price: 579.00, Cost: 1158.00, Comm 2.90
2019-03-13T00:00:00, SELL EXECUTED, Price: 1000.50, Cost: 99255.78, Comm 327.66
2019-03-13T00:00:00, BUY EXECUTED, Price: 742.50, Cost: 742.50, Comm 1.86
2019-03-13T00:00:00, BUY EXECUTED, Price: 717.00, Cost: 2151.00, Comm 5.38
2019-03-13T00:00:00, BUY EXECUTED, Price: 589.00, Cost: 589.00, Comm 1.47
2019-03-18T00:00:00, SELL EXECUTED, Price: 150.00, Cost: 108460.44, Comm 311.62
2019-03-18T00:00:00, BUY EXECUTED, Price: 698.90, Cost: 2096.70, Comm 5.24
2019-03-20T00:00

2019-06-26T00:00:00, Order Canceled/Margin/Rejected
2019-06-26T00:00:00, BUY EXECUTED, Price: 246.20, Cost: 1231.00, Comm 3.08
2019-06-26T00:00:00, BUY EXECUTED, Price: 230.00, Cost: 1840.00, Comm 4.60
2019-06-26T00:00:00, BUY EXECUTED, Price: 137.55, Cost: 3301.20, Comm 8.25
2019-06-26T00:00:00, BUY EXECUTED, Price: 545.00, Cost: 1090.00, Comm 2.73
2019-06-26T00:00:00, BUY EXECUTED, Price: 457.45, Cost: 9149.00, Comm 22.87
2019-07-01T00:00:00, Order Canceled/Margin/Rejected
2019-07-01T00:00:00, BUY EXECUTED, Price: 133.05, Cost: 4390.65, Comm 10.98
2019-07-01T00:00:00, BUY EXECUTED, Price: 538.10, Cost: 2152.40, Comm 5.38
2019-07-03T00:00:00, Order Canceled/Margin/Rejected
2019-07-03T00:00:00, Order Canceled/Margin/Rejected
2019-07-03T00:00:00, Order Canceled/Margin/Rejected
2019-07-03T00:00:00, Order Canceled/Margin/Rejected
2019-07-03T00:00:00, SELL EXECUTED, Price: 235.90, Cost: 1045.94, Comm 1.77
2019-07-03T00:00:00, BUY EXECUTED, Price: 131.50, Cost: 1446.50, Comm 3.62
2019-07-08

2019-11-18T00:00:00, BUY EXECUTED, Price: 907.00, Cost: 2721.00, Comm 6.80
2019-11-18T00:00:00, BUY EXECUTED, Price: 54.50, Cost: 4142.00, Comm 10.36
2019-11-18T00:00:00, BUY EXECUTED, Price: 3.95, Cost: 1544.45, Comm 3.86
2019-11-18T00:00:00, BUY EXECUTED, Price: 414.75, Cost: 93318.75, Comm 233.30
2019-11-20T00:00:00, BUY EXECUTED, Price: 887.00, Cost: 7096.00, Comm 17.74
2019-11-20T00:00:00, BUY EXECUTED, Price: 55.65, Cost: 2281.65, Comm 5.70
2019-11-25T00:00:00, BUY EXECUTED, Price: 55.15, Cost: 1433.90, Comm 3.58
2019-12-02T00:00:00, BUY EXECUTED, Price: 543.25, Cost: 3802.75, Comm 9.51
2019-12-02T00:00:00, BUY EXECUTED, Price: 154.05, Cost: 2464.80, Comm 6.16
2019-12-04T00:00:00, BUY EXECUTED, Price: 534.00, Cost: 1602.00, Comm 4.00
2019-12-04T00:00:00, BUY EXECUTED, Price: 115.05, Cost: 7363.20, Comm 18.41
2019-12-04T00:00:00, BUY EXECUTED, Price: 152.25, Cost: 913.50, Comm 2.28
2019-12-09T00:00:00, BUY EXECUTED, Price: 520.00, Cost: 2080.00, Comm 5.20
2019-12-09T00:00:00, BUY 

2020-06-03T00:00:00, BUY EXECUTED, Price: 89.80, Cost: 34124.00, Comm 85.31
2020-06-17T00:00:00, SELL EXECUTED, Price: 377.95, Cost: 134596.46, Comm 328.82
2020-06-17T00:00:00, BUY EXECUTED, Price: 92.95, Cost: 21936.20, Comm 54.84
2020-06-22T00:00:00, BUY EXECUTED, Price: 94.10, Cost: 752.80, Comm 1.88
2020-06-24T00:00:00, SELL EXECUTED, Price: 475.25, Cost: 117488.14, Comm 314.85
2020-06-29T00:00:00, BUY EXECUTED, Price: 83.75, Cost: 32243.75, Comm 80.61
2020-06-29T00:00:00, BUY EXECUTED, Price: 96.40, Cost: 4048.80, Comm 10.12
2020-06-29T00:00:00, BUY EXECUTED, Price: 83.95, Cost: 4533.30, Comm 11.33
2020-07-01T00:00:00, BUY EXECUTED, Price: 81.10, Cost: 2433.00, Comm 6.08
2020-07-01T00:00:00, BUY EXECUTED, Price: 79.00, Cost: 2054.00, Comm 5.13
2020-07-06T00:00:00, BUY EXECUTED, Price: 83.15, Cost: 166.30, Comm 0.42
2020-07-08T00:00:00, BUY EXECUTED, Price: 92.85, Cost: 6778.05, Comm 16.95
2020-07-08T00:00:00, BUY EXECUTED, Price: 82.00, Cost: 1558.00, Comm 3.90
2020-07-13T00:00:00

2020-11-17T00:00:00, BUY EXECUTED, Price: 28.40, Cost: 3493.20, Comm 8.73
2020-11-17T00:00:00, BUY EXECUTED, Price: 431.40, Cost: 4314.00, Comm 10.79
2020-11-17T00:00:00, BUY EXECUTED, Price: 9.10, Cost: 2629.90, Comm 6.57
2020-11-18T00:00:00, BUY EXECUTED, Price: 28.30, Cost: 849.00, Comm 2.12
2020-11-18T00:00:00, BUY EXECUTED, Price: 430.00, Cost: 860.00, Comm 2.15
2020-11-23T00:00:00, BUY EXECUTED, Price: 421.00, Cost: 842.00, Comm 2.10
2020-11-23T00:00:00, BUY EXECUTED, Price: 423.00, Cost: 2961.00, Comm 7.40
2020-11-25T00:00:00, SELL EXECUTED, Price: 86.50, Cost: 119244.80, Comm 343.84
2020-11-25T00:00:00, SELL EXECUTED, Price: 431.50, Cost: 9762.12, Comm 30.21
2020-11-25T00:00:00, BUY EXECUTED, Price: 428.80, Cost: 1286.40, Comm 3.22
2020-12-01T00:00:00, SELL EXECUTED, Price: 1105.05, Cost: 12934.53, Comm 55.25
2020-12-01T00:00:00, BUY EXECUTED, Price: 420.00, Cost: 6300.00, Comm 15.75
2020-12-02T00:00:00, SELL EXECUTED, Price: 577.65, Cost: 167328.45, Comm 398.58
2020-12-02T00:0

In [225]:
strat = results[0]
pyfoliozer = strat.analyzers.getbyname('pyfolio')

In [226]:
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()

In [227]:
pf.show_perf_stats(returns)

Start date,2010-01-04
End date,2020-12-31
Total months,130
,Backtest
Annual return,17.689%
Cumulative returns,484.613%
Annual volatility,16.87%
Sharpe ratio,1.05
Calmar ratio,0.46
Stability,0.91
Max drawdown,-38.434%
